In [123]:
from warnings import filterwarnings

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import model_selection
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import mean_absolute_error as mae

In [124]:
filterwarnings("ignore")

### Задание 2.7

Объёмная и содержательная практика у нас ещё впереди, но в качестве разминки давайте поработаем с уже известным вам датасетом о вине, который можно скачать здесь.

Ранее вы обучали на данных только один алгоритм, а теперь мы попробуем сравнить несколько.

* Подготовьте данные к классификации. Условно разделите вино на хорошее и плохое. Хорошим вином будем называть то, параметр quality которого — 6 и более.
* Сравните несколько методов классификации: логистическую регрессию, дерево решений и бэггинг. Это позволит вам увидеть, как меняется качество в зависимости от выбора того или иного алгоритма.
* Разделите выборку на обучающую и тестовую в соотношении 70/30, в качестве значения параметра random_state возьмите число 42.
* Для начала обучите два классификатора: логистическую регрессию (с параметрами по умолчанию) и дерево решений (random_state = 42, максимальная глубина — 10).

In [125]:
data_wine = pd.read_csv('data/wineQualityReds.csv', index_col=0)
data_wine.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
1,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
3,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
4,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
5,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [126]:
data_wine['horosh'] = data_wine['quality'].apply(lambda x : 1 if x > 5 else 0)
# data_wine = data_wine.drop('quality', axis=1)
data_wine = data_wine.drop(columns=["quality"])
data_wine.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,horosh
1,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
3,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
4,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
5,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [127]:
X = data_wine.drop('horosh', axis=1) 
y = data_wine['horosh']

# Формируем обучающую и тестовую выборки
# X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3, random_state=42)
print('Train shape: {}'.format(X_train.shape))
print('Test shape: {}'.format(X_test.shape))

Train shape: (1119, 11)
Test shape: (480, 11)


In [128]:
dt_log = LogisticRegression()  #  (random_state=42, criterion='entropy') LogisticRegression
dt_log.fit(X_train, y_train)

LogisticRegression()

In [129]:
#Делаем предсказание класса
y_pred_train = dt_log.predict(X_train)
y_pred_test = dt_log.predict(X_test)

#Выводим отчет о метриках
print(' __________ Тренировочная выборка ___________')
print(metrics.classification_report(y_train, y_pred_train, digits=4))
print('')
print(' __________ Тестовая выборка ________________')
print(metrics.classification_report(y_test, y_pred_test, digits=4))

 __________ Тренировочная выборка ___________
              precision    recall  f1-score   support

           0     0.7389    0.7514    0.7451       531
           1     0.7720    0.7602    0.7661       588

    accuracy                         0.7560      1119
   macro avg     0.7555    0.7558    0.7556      1119
weighted avg     0.7563    0.7560    0.7561      1119


 __________ Тестовая выборка ________________
              precision    recall  f1-score   support

           0     0.6878    0.7136    0.7005       213
           1     0.7645    0.7416    0.7529       267

    accuracy                         0.7292       480
   macro avg     0.7261    0.7276    0.7267       480
weighted avg     0.7304    0.7292    0.7296       480



In [130]:
# Инициализируем модель дерева решений с максимальной глубиной 10 и обучаем её
dt = DecisionTreeClassifier(
    max_depth=10, #глубина    
    random_state=42 #генератор случайных чисел
)
dt.fit(X_train, y_train)
# Визуализируем дерево решений в виде графа
# tree.plot_tree(
#     decision_tree=dt, #дерево решений 
#     feature_names=X_train.columns, #имена факторов
#     filled=True, #расцветка
#     impurity=True, #отображать ли неоднородность в вершинах
# )

DecisionTreeClassifier(max_depth=10, random_state=42)

In [131]:
#Делаем предсказание класса
y_pred_train_t = dt.predict(X_train)
y_pred_test_t = dt.predict(X_test)

#Выводим отчет о метриках
print(' __________ Тренировочная выборка ___________')
print(metrics.classification_report(y_train, y_pred_train_t, digits=4))
print('')
print(' __________ Тестовая выборка ________________')
print(metrics.classification_report(y_test, y_pred_test_t, digits=4))

 __________ Тренировочная выборка ___________
              precision    recall  f1-score   support

           0     0.9312    0.9435    0.9373       531
           1     0.9484    0.9371    0.9427       588

    accuracy                         0.9401      1119
   macro avg     0.9398    0.9403    0.9400      1119
weighted avg     0.9402    0.9401    0.9401      1119


 __________ Тестовая выборка ________________
              precision    recall  f1-score   support

           0     0.7552    0.6808    0.7160       213
           1     0.7639    0.8240    0.7928       267

    accuracy                         0.7604       480
   macro avg     0.7595    0.7524    0.7544       480
weighted avg     0.7600    0.7604    0.7587       480



In [132]:
# построение бэггинг-моделей
bc = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(
        max_depth = 10, 
        random_state=42), 
    n_estimators=1500, 
    random_state=42)

bc.fit(X_train, y_train)
pr_train = bc.predict(X_train)
pr_test = bc.predict(X_test)
f1_score(pr_test, y_test).round(3)

0.818

* crim_rate — уровень преступности в районе;
* zn — доля участков площадью более 25 000 кв. футов;
* business — уровень развитости бизнеса в районе;
* river — наличие реки в районе;
* nit_oxiden — концентрация оксидов азота в воздухе;
* rooms — среднее число комнат в домах района;
* age — процент домов, построенных до 1940 года;
* dist — расстояние до центров занятости;
* highways_index — индекс доступности крупных дорог;
* tax — средняя ставка налога на имущество;
* pup_per_teac — среднее число учеников на одного учителя;
* lower — процент малообеспеченного населения в районе;
* target — медианная стоимость домов в районе (целевая переменная).

###  Задание 3.4

1. Разбейте набор данных на обучающую и тестовую выборку в соотношении 70/30, при разбиении задайте параметр random_state = 13.

Какое получилось среднее значение медианных цен на обучающей выборке? Ответ округлите до двух знаков после точки-разделителя.

In [133]:
data_hous = pd.read_csv('data/boston.csv', decimal=',')
data_hous.head()

,crim_rate,zn,business,river,nit_oxiden,rooms,age,dist,highways_index,tax,pup_per_teaс,lower,target
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,5.33,36.2


In [134]:
Xh = data_hous.drop('target', axis=1) 
yh = data_hous['target']

# Формируем обучающую и тестовую выборки
X_train_h, X_test_h, y_train_h, y_test_h = model_selection.train_test_split(Xh, yh, test_size=0.3, random_state=13)
print('Train shape: {}'.format(X_train_h.shape))
print('Test shape: {}'.format(X_test_h.shape))


Train shape: (354, 12)
Test shape: (152, 12)


In [135]:
y_train_h.mean()

22.773163841807918

In [136]:
lin_reg_h = LinearRegression()

lin_reg_h.fit(X_train_h, y_train_h)

LinearRegression()

In [137]:
#Делаем предсказание класса
y_pred_train_h = lin_reg_h.predict(X_train_h)
y_pred_test_h = lin_reg_h.predict(X_test_h)

mae(y_pred_test_h, y_test_h).round(3)


3.722

In [138]:
# Инициализируем модель дерева решений и обучаем её
dt_h = DecisionTreeRegressor(random_state=13)
dt_h.fit(X_train_h, y_train_h)

y_pred_train_th = dt_h.predict(X_train_h)
y_pred_test_th = dt_h.predict(X_test_h)

print(mae(y_pred_train_th, y_train_h).round(3))
print(mae(y_pred_test_th, y_test_h).round(3))

0.0
2.839


In [139]:
# Инициализируем модель дерева решений с максимальной глубиной 3 и обучаем её
dt_3 = DecisionTreeRegressor(
    max_depth=3, #глубина    
    random_state=13 #генератор случайных чисел
)
dt_3.fit(X_train_h, y_train_h)

y_pred_train_3 = dt_3.predict(X_train_h)
y_pred_test_3 = dt_3.predict(X_test_h)

print(mae(y_pred_train_3, y_train_h).round(3))
print(mae(y_pred_test_3, y_test_h).round(3))


3.098
3.913


In [140]:
# Инициализируем модель дерева решений с максимальной глубиной 10 и обучаем её
dt_10 = DecisionTreeRegressor(
    max_depth=10, #глубина    
    random_state=13 #генератор случайных чисел
)
dt_10.fit(X_train_h, y_train_h)

y_pred_train_10 = dt_10.predict(X_train_h)
y_pred_test_10 = dt_10.predict(X_test_h)

print(mae(y_pred_train_10, y_train_h).round(3))
print(mae(y_pred_test_10, y_test_h).round(3))

0.554
2.861


In [141]:
# Инициализируем модель дерева решений с максимальной глубиной 100 и обучаем её
dt_100 = DecisionTreeRegressor(
    max_depth=100, #глубина    
    random_state=13 #генератор случайных чисел
)
dt_100.fit(X_train_h, y_train_h)

y_pred_train_100 = dt_100.predict(X_train_h)
y_pred_test_100 = dt_100.predict(X_test_h)

print(mae(y_pred_train_100, y_train_h).round(3))
print(mae(y_pred_test_100, y_test_h).round(3))

0.0
2.839


In [142]:
# Инициализируем модель дерева решений с максимальной глубиной 500 и обучаем её
dt_500 = DecisionTreeRegressor(
    max_depth=500, #глубина    
    random_state=13 #генератор случайных чисел
)
dt_500.fit(X_train_h, y_train_h)

y_pred_train_500 = dt_500.predict(X_train_h)
y_pred_test_500 = dt_500.predict(X_test_h)

print(mae(y_pred_train_500, y_train_h).round(3))
print(mae(y_pred_test_500, y_test_h).round(3))

0.0
2.839
